In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory
from collections import OrderedDict
from tqdm import tqdm
import datetime

cudnn.benchmark = True
plt.ion()   # interactive mode

In [19]:
num = 0
for filename in os.listdir('D:/data2'):
    entries = os.listdir('D:/data2/' + filename)
    num += len(entries)
print(num)

288


In [20]:
dataset = datasets.ImageFolder('D:/data2', transform=transforms.Compose([
        # transforms.Resize((1280, 1024)),
        transforms.Resize((644, 644)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]))
class_names = dataset.classes
len(class_names)

7

In [21]:
trainds, valds, testds = torch.utils.data.random_split(dataset, [0.4, 0.3, 0.3], generator=torch.Generator().manual_seed(15275))

dataloaders = {'train': torch.utils.data.DataLoader(trainds, batch_size=32,
                                             shuffle=True, num_workers=16), 
               'val': torch.utils.data.DataLoader(valds, batch_size=32,
                                             shuffle=True, num_workers=16), 
               'test': torch.utils.data.DataLoader(testds, batch_size=32,
                                             shuffle=True, num_workers=16)}

dataset_sizes = {'train': len(trainds), 
               'val': len(valds), 
               'test': len(testds)}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [27]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    train_loss = []
    train_acc = []
    valid_loss = []
    valid_acc = []


    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)
            print(datetime.datetime.now())

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in tqdm(dataloaders[phase]):
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    with torch.set_grad_enabled(phase == 'train'):
                        if phase == 'train':
                            outputs, aux_outputs = model(inputs)  # 训练模式下接收两个输出
                        else:
                            outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]
                if phase == 'train':
                    train_loss.append(epoch_loss)
                    train_acc.append(epoch_acc)
                else:
                    valid_loss.append(epoch_loss)
                    valid_acc.append(epoch_acc)

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model, train_loss, train_acc, valid_loss, valid_acc

In [29]:
model_ft = models.inception_v3(weights='IMAGENET1K_V1')
# Here the size of each output sample is set to 5.
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
model_ft.fc = nn.Linear(2048, len(class_names))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [30]:
model_ft

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [33]:
for name, param in model_ft.named_parameters():
    if param.requires_grad:
        print(f"{name} will be updated.")
    else:
        print(f"{name} will not be updated.")

Conv2d_1a_3x3.conv.weight will be updated.
Conv2d_1a_3x3.bn.weight will be updated.
Conv2d_1a_3x3.bn.bias will be updated.
Conv2d_2a_3x3.conv.weight will be updated.
Conv2d_2a_3x3.bn.weight will be updated.
Conv2d_2a_3x3.bn.bias will be updated.
Conv2d_2b_3x3.conv.weight will be updated.
Conv2d_2b_3x3.bn.weight will be updated.
Conv2d_2b_3x3.bn.bias will be updated.
Conv2d_3b_1x1.conv.weight will be updated.
Conv2d_3b_1x1.bn.weight will be updated.
Conv2d_3b_1x1.bn.bias will be updated.
Conv2d_4a_3x3.conv.weight will be updated.
Conv2d_4a_3x3.bn.weight will be updated.
Conv2d_4a_3x3.bn.bias will be updated.
Mixed_5b.branch1x1.conv.weight will be updated.
Mixed_5b.branch1x1.bn.weight will be updated.
Mixed_5b.branch1x1.bn.bias will be updated.
Mixed_5b.branch5x5_1.conv.weight will be updated.
Mixed_5b.branch5x5_1.bn.weight will be updated.
Mixed_5b.branch5x5_1.bn.bias will be updated.
Mixed_5b.branch5x5_2.conv.weight will be updated.
Mixed_5b.branch5x5_2.bn.weight will be updated.
Mixed

In [34]:
model_ft, train_loss, train_acc, valid_loss, valid_acc = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=70)

Epoch 0/69
----------
2025-03-03 09:36:08.323312


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [02:41<00:00, 40.30s/it]


train Loss: 1.4368 Acc: 0.4828


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:49<00:00, 16.60s/it]


val Loss: 1.4024 Acc: 0.5465

Epoch 1/69
----------
2025-03-03 09:39:39.604712


...

val Loss: 0.2087 Acc: 0.9419

Epoch 69/69
----------
2025-03-03 13:35:30.165779


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [02:46<00:00, 41.74s/it]


train Loss: 0.0162 Acc: 1.0000


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:50<00:00, 16.87s/it]


val Loss: 0.2047 Acc: 0.9535

Training complete in 242m 60s
Best val Acc: 0.953488


In [35]:
def test_model(model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(testds)
    correct = 0.0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloaders['test']:
            X= X.to(device)
            outputs = model(X)
            _, preds = torch.max(outputs, 1)
            # print('===================')
            # print(preds)
            # print(y.data)
            correct += torch.sum(preds == y.data)

    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%")

test_model(model_ft, criterion)

Test Error: 
 Accuracy: 91.9%
